# DX 704 Week 7 Project

This week's project will investigate issues in a quadcopter controller based using a linear quadratic regulator.
You will start with an existing model of the system and logs from a quadcopter based on it, investigate discrepancies, and ultimately train a new model of the system dynamics.

The full project description and a template notebook are available on GitHub: [Project 7 Materials](https://github.com/bu-cds-dx704/dx704-project-07).


## Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Introduction

You've just joined a drone startup.
On your first day, you join your new team to watch a test flight for a new quadcopter prototype.
Watching the prototype fly, the team comments that it is not as smooth as usual and suspects that something is off in the controller.
They provide you a copy of the dynamics model and log data from the prototype to investigate.

The quadcopter control model is a slightly more sophisticated version of the 1D quadcopter problem previously considered.

The state vector $\mathbf{x}_t$ now includes an acceleration component, and the action vector now has a servo control for the throttle instead of a raw force component.
\begin{array}{rcl}
\mathbf{x}_t & = & \begin{bmatrix} x_t \\ v_t \\ a_t \end{bmatrix} \\
\mathbf{u_t} & = & \begin{bmatrix} u_t \end{bmatrix}
\end{array}

## Part 1: Reconstruct the Control Matrix

You are provided the dynamics model in the files `model-A.tsv`, `model-B.tsv`, `cost-Q.tsv` and `cost-R.tsv`.
Recompute the control matrix $\mathbf{K}$ to be used in the infinite horizon linear quadratic regulator problem.

In [47]:
# YOUR CHANGES HERE

import pandas as pd 
import numpy as np

from scipy.linalg import solve_discrete_are
from sklearn.linear_model import LinearRegression


In [ ]:
# Load matrices in but need to change the values and dtype of the variable to be float such that we can apply it to our function later on 

A = pd.read_csv("model-A.tsv", sep="\t", header=0, index_col=0).apply(pd.to_numeric).to_numpy(dtype=float)
B = pd.read_csv("model-B.tsv", sep="\t", header=0, index_col=0).apply(pd.to_numeric).to_numpy(dtype=float)
Q = pd.read_csv("cost-Q.tsv", sep="\t", header=0, index_col=0).apply(pd.to_numeric).to_numpy(dtype=float)
R = pd.read_csv("cost-R.tsv", sep="\t", header=0, index_col=0).apply(pd.to_numeric).to_numpy(dtype=float)

convert to numpy arrays

In [ ]:
A

array([[1., 1., 0.],
       [0., 1., 1.],
       [0., 0., 1.]])

In [ ]:
B

array([[0.],
       [0.],
       [1.]])

In [ ]:
Q

array([[5., 0., 0.],
       [0., 1., 0.],
       [0., 0., 2.]])

In [ ]:
R

array([[5.]])

In [ ]:
print(A.shape, B.shape, Q.shape, R.shape)

(3, 3) (3, 1) (3, 3) (1, 1)


In [ ]:
print(A.dtype, B.dtype, Q.dtype, R.dtype)

float64 float64 float64 float64


In [ ]:
# solve using scipy library solve_discrete_are

M = solve_discrete_are(A, B, Q, R)

In [ ]:
K = np.linalg.inv(B.T @ M @ B + R) @ (B.T @ M @ A)

Save $\mathbf{K}$ in a file "control-K-intended.tsv" with columns x, v and a.

In [41]:
# YOUR CHANGES HERE

pd.DataFrame(K, columns=["x", "v", "a"]).to_csv("control-K-intended.tsv", sep="\t", index=False)

Submit "control-K-intended.tsv" in Gradescope.

## Part 2: Recompute the Actions for the Logged States

You get access to the log data for the prototype as the file "qc-log.tsv".
It conveniently saves all the state and actions made.
Recompute the actions based on your $\mathbf{K}$ matrix computed in part 1.

In [43]:
# YOUR CHANGES HERE

K = pd.read_csv("control-K-intended.tsv", sep="\t").to_numpy(dtype=float)
log = pd.read_csv("qc-log.tsv", sep="\t")

In [44]:
X = log[["x", "v", "a"]].to_numpy(dtype=float)

In [ ]:
# u = -K x
U = (-K @ X.T).ravel()

Save your computed actions as "qc-check.tsv" with columns "index" and "u_check".

In [46]:
# YOUR CHANGES HERE

pd.DataFrame({"index": np.arange(len(U)), "u_check": U}).to_csv(
    "qc-check.tsv", sep="\t", index=False
)

Submit "qc-check.tsv" in Gradescope.

## Part 3: Reverse Engineer the Actual Control Matrix

Now that you have found a systematic difference between your computed actions and the logged actions, estimate $
$, the control matrix that was actually used to choose actions in the prototype.

Hint: With a linear quadratic regulator, the optimal actions are always linear combinations of the state that are calculated using the control matrix.
You can use linear regression to reverse-engineer the coefficients in the control matrix.

In [48]:
# YOUR CHANGES HERE

log = pd.read_csv("qc-log.tsv", sep="\t")

In [49]:
X = log[["x", "v", "a"]].to_numpy(dtype=float)
u = log["u"].to_numpy(dtype=float)

In [50]:
model = LinearRegression(fit_intercept=False)
model.fit(X, u)

,fit_intercept,False
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


In [52]:
K_actual = -model.coef_.reshape(1, -1)

Save $\mathbf{K}_{\mathrm{actual}}$ in "control-K-actual.tsv" with the same format as "control-K-intended.tsv".

In [54]:
# YOUR CHANGES HERE

pd.DataFrame(K_actual, columns=["x", "v", "a"]).to_csv(
    "control-K-actual.tsv", sep="\t", index=False
)

Submit "actual-K.tsv" in Gradescope.

## Part 4: Recompute the System Dynamics from the Log Data

On further investigation, it turns out that the control matrix $\mathbf{K}$ in the prototype was modified to compensate for state prediction errors.
You would like to recompute the $\mathbf{A}$ and $\mathbf{B}$ matrices using the log data since they are used to predict the next states.
However, since the action vector $\mathbf{u}_t$ is linearly dependent on the state via $\mathbf{u}_t=-\mathbf{K} \mathbf{x}_t$, you need a new data set so you can separate the effects of the $\mathbf{A}$ and $\mathbf{B}$ matrices.
Your co-workers kindly provide a new file "qc-train.tsv" where noise was added to each action.
Estimate the true $\mathbf{A}$ and $\mathbf{B}$ matrices based on this file.

In [55]:
# YOUR CHANGES HERE

train = pd.read_csv("qc-train.tsv", sep="\t")


In [ ]:
# states and actions from training data
X_t   = train[["x", "v", "a"]].to_numpy(dtype=float)[:-1]
X_tp1 = train[["x", "v", "a"]].to_numpy(dtype=float)[1:]
U_t   = train[["u"]].to_numpy(dtype=float)[:-1]

In [ ]:
# feature matrix
feat_M = np.hstack([X_t, U_t])

In [58]:
# given the noise we have in the training data we can use least squares to help minimize errors
x, _, _, _ = np.linalg.lstsq(feat_M, X_tp1, rcond=None)

In [59]:
A_new = x[:3, :].T 
B_new = x[3:, :].T 

Save $\mathbf{A}$ and $\mathbf{B}$ to "model-A-new.tsv" and "model-B-new.tsv" respectively.

In [60]:
# YOUR CHANGES HERE

pd.DataFrame(A_new, columns=["x","v","a"]).to_csv("model-A-new.tsv", sep="\t", index=False)
pd.DataFrame(B_new, columns=["u"]).to_csv("model-B-new.tsv", sep="\t", index=False)

Submit "model-A-new.tsv" and "model-B-new.tsv" in Gradescope.

## Part 5: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.
You do not need to provide code for data collection if you did that by manually.

## Part 6: Acknowledgements

If you discussed this assignment with anyone, please acknowledge them here.
If you did this assignment completely on your own, simply write none below.

If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for. If you did not use any other libraries, simply write none below.

If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy. If you did not use any generative AI tools, simply write none below.